# Sparse matrix respresentation

In this notebook will introduce you to the concepts of **sparse matrix representation**. This is a useful way to numerically represent matrices when the amount of zeros is really large. It ensures that such large matrices can still be stored into memory without causing errors. We will focus on the different sparse matrix formats that exist, and a Python library (called *scipy sparse*) that allows you to use sparse matrices.

## The issue

In the code below, we create a square matrix with only one nonzero entry. Namely, there will be a value of '2' in the second entry of the first row.

In [13]:
import numpy as np
n = 100 #dimension of the matrix
A = np.zeros((n,n)) #start with an all-zero matrix
A[0, 1] = 2 #Change the value in the first row and second column to a 1

A

array([[0., 2., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

As expected, running this code is not difficult. There is nothing big going on. However, try changing the value of $n$ to e.g. $n = 1000000$, and you will see an interesting error pop up. Apparently, a huge amount of RAM is being using to store such a trivial matrix. What is going wrong here? In essence, Python is trying to allocate a piece of memory for every entry of the matrix; even the zeros. We call this **full-format** storage of the matrix. Full-format storage wastes a lot of space to zeros, causing the memory to be completely flooded. The idea of sparse matrix representation is to only store non-zero values. In case of our trivial matrix, this will reduce memory allocation significantly.

## Full-format storage

Memory in your computer is sequential. It can be modelled as a big sequence of stored numbers. The main problem with matrices, is that they are usually not considered to be a sequence. Matrices are usually represented as a block. So, how do we turn this block into a sequence? In full-format storage, we give the entire matrix one adress in memory, and then store the following sequentially at the adress:
1. We first store the dimensions of the matrix.
2. We then store the data in the matrix column by column or row by row.

The two examples below show how full-format matrices are stored in memory.

In [11]:
#Row-by-row storage
B = np.array([[1,2,3],[4,5,6],[7,8,9]])
print(B)

#In memory, first store the dimensions
print("The dimensions are {} by {}".format(B.shape[0], B.shape[1]))
#Then, store the data row by row
print("The data in the matrix is: {}".format(B.flatten()))

[[1 2 3]
 [4 5 6]
 [7 8 9]]
The dimensions are 3 by 3
The data in the matrix is: [1 2 3 4 5 6 7 8 9]


In [12]:
#Column-by-column storage
B = np.array([[1,2,3],[4,5,6],[7,8,9]])
print(B)

#In memory, first store the dimensions
print("The dimensions are {} by {}".format(B.shape[0], B.shape[1]))
#Then, store the data row by row
print("The data in the matrix is: {}".format(B.flatten("F")))

[[1 2 3]
 [4 5 6]
 [7 8 9]]
The dimensions are 3 by 3
The data in the matrix is: [1 4 7 2 5 8 3 6 9]


For Python, row by row storage is default. Note this is not the case for all programming languages. MatLab, for example, uses column by column storage is its default. This difference might cause issues e.g. when using linear indexing for matrices.

## COO sparse storage

All sparse storage of matrices is handled by the *Scipy* package in Python. The simplest format to create a sparse matrix in Python is by using the **COO (coordinate) format**. The idea of this matrix storage method is simple. You store:
1. The dimensions of the matrix.
2. A vector containing the row indices of the nonzero entries.
3. A vector containing the column indices of the nonzero entries.
4. A vector with the nonzero entries.

Below is an example that shows how we would define the 1000000 by 1000000 matrix *A* from before in COO-format.

In [15]:
import scipy as sp

#Data for COO-storage
n, m = 1000000, 1000000
row = np.array([0])
col = np.array([1])
data = np.array([2])
#Creating the matrix
A = sp.sparse.coo_matrix((data, (row, col)), shape = (n, m))
#Printing all-nonzero values and their coordinates
print(A)

  (0, 1)	2


Be aware that COO-format is usually only used to create sparse matrices. It is not really possible to do any operations with these matrices. You would need to convert COO-matrices to other sparse formats to do operations with them. Be aware never to convert sparse matrices to dense formats, since this will again use up all memory, defeating the point of sparse storage, as the example below shows.

In [16]:
#Go from sparse format to full-format
A.todense()

MemoryError: Unable to allocate 3.64 TiB for an array with shape (1000000, 1000000) and data type int32

## CSC and CSR sparse storage

Two sparse storgage formats that produce matrices that allow for easy manipulation are the **CSC (compressed sparse column) format** and the **CSR (compressed sparse row) format**. In case of the CSC-format, the following data is supplied to memory:
1. The dimensions of the matrix
2. A vector $v$ storing the row-indices of each nonzero entry.
3. A vector storing at which index of $v$ each column starts.
4. A vector containing all nonzero entries of the matrix.

Below is an example where we create our matrix *A* in CSC-format.

In [20]:
#Data for CSC-storage
n, m = 1000000, 1000000
row = np.array([0]) 
colstart = np.ones(m + 1)
colstart[[0,1]] = [0, 0]
data = np.array([2])
#Creating the matrix
A = sp.sparse.csc_matrix((data, row, colstart), shape = (n, m))
#Printing all-nonzero values and their coordinates
print(A)

  (0, 1)	2


Note that in the example above `colstart` needs to be a vector of size 1000001 since it needs to indicate for every column where it starts and ends with respect to the row index array. `colstart` begins with two zeros, since the first column is empty. The second entry in `colstart` is zero and the third is one. This means that the second column starts at index zero of `row` and ends at index one. This end index is not considered part of the column. Hence, this shows there is one nonzero entry in the second column. Its row-index is given by `row[0] = 0` and its value by `data[0] = 2`. Finally, all the other values in `colstart` are one, indicating that all other columns have that their nonzero values start at index one of `row` (which does not exist) and also end there, meaning all the other columns only contain zeros.

You might have noticed that the CSC-format to define a matrix is pretty cumbersome. Hence, usually you want to define your matrix in COO-format and convert to CSC-format to do calculations with it. The example below shows how you would do that.

In [23]:
#Data for COO-storage
n, m = 1000000, 1000000
row = np.array([0])
col = np.array([1])
data = np.array([2])
#Creating the matrix in COO-format
A = sp.sparse.coo_matrix((data, (row, col)), shape = (n, m))
#Converting to CSC-format
A = A.tocsc()

A #Showing that the format is indeed CSC

<1000000x1000000 sparse matrix of type '<class 'numpy.intc'>'
	with 1 stored elements in Compressed Sparse Column format>

CSR-format is similar to CSC-format, but in this case the role of the two index arrays flip. The following information needs to be provided:

1. The dimensions of the matrix
2. A vector $v$ storing the **column**-indices of each nonzero entry.
3. A vector storing at which index of $v$ each **row** starts.
4. A vector containing all nonzero entries of the matrix.

Again we will give our matrix *A* as an example. Try to explain yourself why this code creates the correct format.

In [25]:
#Data for CSR-storage
n, m = 1000000, 1000000
column = np.array([1]) 
rowstart = np.ones(m + 1)
rowstart[0] = 0
data = np.array([2])
#Creating the matrix
A = sp.sparse.csr_matrix((data, column, rowstart), shape = (n, m))
#Printing all-nonzero values and their coordinates
print(A)

  (0, 1)	2


Below is one more example where we create a sparse matrix in CSR-format. We first give the matrix in full-format, then define the matrix is CSR-format, and finally check whether these two matrices are the same. Try to explain again why this way of defining the CSR-format matrix is correct.

In [28]:
#Full-format matrix
Mdense = np.array([[0,2,0,0,3],[1,0,0,6,3],[0,0,0,0,0],[0,0,0,9,2]])
print(Mdense)

#Sparse CSR-matrix
n, m = 4, 5
column = np.array([1, 4, 0, 3, 4, 3, 4])
rowstart = np.array([0, 2, 5, 5, 7])
data = np.array([2, 3, 1, 6, 3, 9, 2])
Msparse = sp.sparse.csr_matrix((data, column, rowstart), shape = (n, m))
print(Msparse)

#Checking whether Mdense and Msparse are the same
print(Mdense == Msparse)

[[0 2 0 0 3]
 [1 0 0 6 3]
 [0 0 0 0 0]
 [0 0 0 9 2]]
  (0, 1)	2
  (0, 4)	3
  (1, 0)	1
  (1, 3)	6
  (1, 4)	3
  (3, 3)	9
  (3, 4)	2
[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]]


**Exercise.** The first code block below generates a random full-format matrix. Use the second code block below to implement it as a sparse matrix. Experiment with COO-, CSC- and CSR-formats to get more familiar with them. You can also try to implement code that automatically turns a full-format matrix into a sparse format.

In [43]:
#Creating a random matrix in full-format
[n, m] = np.random.default_rng().integers(2, high = 7, size = 2)
Mdense = np.random.default_rng().choice(3, size = (n, m), p = [0.5, 0.4, 0.1])
print("The dense matrix is:")
print(Mdense)

The dense matrix is:
[[1 0 0 2 0 1]
 [1 0 2 1 1 1]
 [0 0 1 0 2 1]]


In [44]:
#Use this code block to make a sparse matrix (called Msparse) that is the same as Mdense


#Check whether you are correct
print(Mdense == Msparse)

False


## Some notes on matrix operations and auxiliary functions

When you are doing matrix operations with sparse matrices, the most important thing to keep in mind is to *never convert the matrix back to full-format*. In general this means that you should avoid *NumPy* funcitons as much as you can, since they only work with full-format matrices. The *Scipy.sparse* package has many of the same options as the *NumPy* package, but does not need full-format matrices. The example below shows that.

In [56]:
#Creating a sparse matrix
n, m = 1000000, 1000000
column = np.array([1]) 
rowstart = np.ones(m + 1)
rowstart[0] = 0
data = np.array([2])
A = sp.sparse.csr_matrix((data, column, rowstart), shape = (n, m))

#Calculating the frobenius norm of this matrix in a sparse way (the answer should be 2)
out = sp.sparse.linalg.norm(A)
print("The frobenius norm of A is: {}".format(out))

#Calculating the frobenius norm of this matrix using Numpy
out = np.linalg.norm(A)
print("The frobenius norm of A is: {}".format(out))

The frobenius norm of A is: 2.0
The frobenius norm of A is: 


When using functions on sparse matrices, be aware that certain functions work much better in certain formats due to how things are stored in memory. Some examples are:
- COO-matrices are not really suited for arithmatic operations and slicing. They need to be converted to a CSC- or CSR-format.
- CSR-matrices are well suited for row selection, but terrible at column selection. This is because in CSR-format row data is placed adjacent in memory, while column data is scattered all over memory.
- CSC-matrices are well suited for column selection, but terrible at row selection. This is because in CSC-format column data is placed adjacent in memory, while row data is scattered all over memory.

The code block below should show these three examples. Be aware that it can take up to one minute to run this block. If you want a quicker, but less convincing output, remove one of the zeros from $n$ and $m$.

In [65]:
#The matrix A contains all zeros except for the first row, which only contains ones.
n, m = 1000000000, 1000000000
row = np.zeros(m)
col = np.arange(n)
data = np.ones_like(col)
A = sp.sparse.coo_matrix((data, (row, col)), shape = (n, m)) 

#Try to select the second column in COO-form
try:
    print(A[:,1])
except:
    print("I could not extract the second column from A...")
    
#Try to select the second column in CSR-form and time the result
import time
Acsr = A.tocsr()
t1 = time.time()
print("The second column of A is: {}".format(Acsr[:,1]))
print("This operation took me {} seconds in CSR-format".format(time.time() - t1))

#Try to select the second column in CSC-form and time the result
Acsc = A.tocsc()
t1 = time.time()
print("The second column of A is: {}".format(Acsc[:,1]))
print("This operation took me {} seconds in CSC-format".format(time.time() - t1))

I could not extract the second column from A...
The second column of A is:   (0, 0)	1
This operation took me 25.35190176963806 seconds in CSR-format
The second column of A is:   (0, 0)	1
This operation took me 0.04050087928771973 seconds in CSC-format


Finally, in the course we will be working with the *NetworkX* package a lot. If you ever extract things like the adjacency matrix from this package, then it will be given in a sparse format. The example below shows this.

In [71]:
import networkx as nx

G = nx.fast_gnp_random_graph(10**6, 2*10**(-6)) #Create an Erdos-Renyi random graph.
Adj = nx.adjacency_matrix(G)
print("Is the adjacency matrix sparse? {}".format(sp.sparse.issparse(Adj))) #Checks if the matrix is sparse

C:\Users\s159636\AppData\Local\Temp\ipykernel_9112\1545770259.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj = nx.adjacency_matrix(G)


Is the adjacency matrix sparse? True
